In [1]:
import numpy as np
import pandas as pd
from random import choices

import warnings
warnings.filterwarnings("ignore")

from sklearn import metrics 
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from IPython.display import display

from bokeh.io import push_notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row

from imblearn.over_sampling import SMOTE
from category_encoders import WOEEncoder
import csv

output_notebook()

Using TensorFlow backend.


Loading BokehJS ...

In [2]:
# klienci = pd.read_csv("klienci.txt", encoding = "cp1250", index_col = "numer", decimal=".", sep = ",")
# klienci.to_pickle("klienci.pkl")
# klienci_final = pd.read_csv("klienci_final.txt", encoding = "cp1250", decimal=".", sep = ",")
# klienci_final.set_index('numer', inplace=True)
# klienci_final.to_pickle("klienci_final.pkl")

In [3]:
klienci = pd.read_pickle("klienci.pkl")
klienci.head()

,wiek,plec,wyksztalcenie,miejsce,kwota,wojewodztwo,wysylka_oferty,zakup
numer,,,,,,,,
1,32,1,srednie,wies,110.810558,dolsl,0,0
2,26,1,wyzsze,m100_500,100.857513,zachpom,0,0
3,20,0,srednie,m100_500,66.199017,zachpom,0,0
4,18,1,policealne,m100_500,77.017056,sląs,1,0
5,26,1,srednie,m100_500,137.221167,zachpom,0,0


In [4]:
def rows_number(dataset):
    total_rows = dataset.shape[0]
    print('Liczba rekordów w zbiorze: '+ str(total_rows))

In [5]:
rows_number(klienci)

Liczba rekordów w zbiorze: 10034


In [6]:
klienci_final = pd.read_pickle("klienci_final.pkl")
klienci_final.head()

,wiek,plec,wyksztalcenie,miejsce,kwota,wojewodztwo
numer,,,,,,
1,35,0,podstawowe,wies,97.965209,zachpom
2,26,1,podstawowe,wies,38.377882,dolsl
3,31,0,srednie,m10_100,136.401297,dolsl
4,25,0,policealne,m100_500,118.728067,dolsl
5,18,1,srednie,m_500,79.311481,dolsl


In [7]:
rows_number(klienci_final)

Liczba rekordów w zbiorze: 100895


In [8]:
dict = {1 : 'k', 0: 'm'} 
klienci['plec'].replace(dict, inplace=True)
dict = {1 : 'tak', 0: 'nie'} 
klienci['wysylka_oferty'].replace(dict, inplace=True)
klienci.head()

,wiek,plec,wyksztalcenie,miejsce,kwota,wojewodztwo,wysylka_oferty,zakup
numer,,,,,,,,
1,32,k,srednie,wies,110.810558,dolsl,nie,0
2,26,k,wyzsze,m100_500,100.857513,zachpom,nie,0
3,20,m,srednie,m100_500,66.199017,zachpom,nie,0
4,18,k,policealne,m100_500,77.017056,sląs,tak,0
5,26,k,srednie,m100_500,137.221167,zachpom,nie,0


In [9]:
dict = {1 : 'k', 0: 'm'} 
klienci_final['plec'].replace(dict, inplace=True)
klienci_final.head()

,wiek,plec,wyksztalcenie,miejsce,kwota,wojewodztwo
numer,,,,,,
1,35,m,podstawowe,wies,97.965209,zachpom
2,26,k,podstawowe,wies,38.377882,dolsl
3,31,m,srednie,m10_100,136.401297,dolsl
4,25,m,policealne,m100_500,118.728067,dolsl
5,18,k,srednie,m_500,79.311481,dolsl


In [10]:
target = 'zakup'
cat_col = ['plec', 'wyksztalcenie', 'miejsce', 'wojewodztwo', 'wysylka_oferty']
num_col = ['wiek', 'kwota']
x_col = ['wiek', 'plec', 'wyksztalcenie', 'miejsce', 'kwota', 'wojewodztwo', 'wysylka_oferty']

In [11]:
def mean_by(dataframe, by, col_mean):
    group_by = dataframe.groupby(by)[col_mean].mean()
    return pd.DataFrame(group_by)

In [12]:
wyksztalcenie = mean_by(klienci, 'wyksztalcenie', target)
display(wyksztalcenie)

,zakup
wyksztalcenie,
brak,0.013289
podstawowe,0.009492
policealne,0.018272
srednie,0.113360
wyzsze,0.364529


In [13]:
for i in cat_col: 
    tab1 = mean_by(klienci, i, target)
    label = tab1.index.values
    values = tab1['zakup']  
    p = figure(x_range=label, plot_height=250, title="Prawdopodobieństwo zakupu względem kategorii zmiennej: "+ i)
    p.vbar(x=label, top=values, width=0.9)
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    show(p)

In [14]:
var = 'wiek'
bins=30
target_0 = klienci.loc[klienci[target] == 0]
hist, edges = np.histogram(target_0[var], density=True, bins=bins)
p1 = figure(title = "Rozkład zmiennej: "+var+" dla "+target+"=0", plot_width=400, plot_height=400)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p1.y_range.start = 0

target_1 = klienci.loc[klienci[target] == 1]
hist, edges = np.histogram(target_1[var], density=True, bins=bins)
p2 = figure(title = "Rozkład zmiennej: "+var+" dla "+target+"=1", plot_width=400, plot_height=400)
p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p2.y_range.start = 0

show(row(p1, p2))

In [15]:
var = 'kwota'
bins=30
target_0 = klienci.loc[klienci[target] == 0]
hist, edges = np.histogram(target_0[var], density=True, bins=bins)
p1 = figure(title = "Rozkład zmiennej: "+var+" dla "+target+"=0", plot_width=400, plot_height=400)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p1.y_range.start = 0

target_1 = klienci.loc[klienci[target] == 1]
hist, edges = np.histogram(target_1[var], density=True, bins=bins)
p2 = figure(title = "Rozkład zmiennej: "+var+" dla "+target+"=1", plot_width=400, plot_height=400)
p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p2.y_range.start = 0

show(row(p1, p2))

In [16]:
klienci[target].value_counts()

0    8717
1    1317
Name: zakup, dtype: int64

In [17]:
klienci_dummies = pd.get_dummies(klienci, columns = cat_col, prefix = cat_col)

In [18]:
y_klienci = klienci_dummies[target]
x_klienci = klienci_dummies.drop(target, axis=1)
x_klienci_names = x_klienci.columns
x_klienci.head()

,wiek,kwota,plec_k,plec_m,wyksztalcenie_brak,wyksztalcenie_podstawowe,wyksztalcenie_policealne,wyksztalcenie_srednie,wyksztalcenie_wyzsze,miejsce_m0_10,...,wojewodztwo_podlas,wojewodztwo_pomo,wojewodztwo_sląs,wojewodztwo_swięt,wojewodztwo_warmaz,wojewodztwo_wielkop,wojewodztwo_zachpom,wojewodztwo_łódz,wysylka_oferty_nie,wysylka_oferty_tak
numer,,,,,,,,,,,,,,,,,,,,,
1,32,110.810558,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,26,100.857513,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
3,20,66.199017,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
4,18,77.017056,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
5,26,137.221167,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [19]:
sm = SMOTE(sampling_strategy= 1)

In [20]:
x_klienci_smote, y_klienci_smote = sm.fit_sample(x_klienci, y_klienci)

In [21]:
y_klienci_smote = pd.DataFrame(y_klienci_smote)
y_klienci_smote.columns = [target]

In [22]:
y_klienci_smote[target].value_counts()

1    8717
0    8717
Name: zakup, dtype: int64

In [23]:
x_klienci_smote = pd.DataFrame(x_klienci_smote)
x_klienci_smote.columns = x_klienci_names
x_klienci_smote.head()

,wiek,kwota,plec_k,plec_m,wyksztalcenie_brak,wyksztalcenie_podstawowe,wyksztalcenie_policealne,wyksztalcenie_srednie,wyksztalcenie_wyzsze,miejsce_m0_10,...,wojewodztwo_podlas,wojewodztwo_pomo,wojewodztwo_sląs,wojewodztwo_swięt,wojewodztwo_warmaz,wojewodztwo_wielkop,wojewodztwo_zachpom,wojewodztwo_łódz,wysylka_oferty_nie,wysylka_oferty_tak
0,32.0,110.810558,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,26.0,100.857513,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,20.0,66.199017,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,18.0,77.017056,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,26.0,137.221167,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [24]:
xy_klienci_smote = x_klienci_smote.join(y_klienci_smote)
xy_klienci_smote.head()

,wiek,kwota,plec_k,plec_m,wyksztalcenie_brak,wyksztalcenie_podstawowe,wyksztalcenie_policealne,wyksztalcenie_srednie,wyksztalcenie_wyzsze,miejsce_m0_10,...,wojewodztwo_pomo,wojewodztwo_sląs,wojewodztwo_swięt,wojewodztwo_warmaz,wojewodztwo_wielkop,wojewodztwo_zachpom,wojewodztwo_łódz,wysylka_oferty_nie,wysylka_oferty_tak,zakup
0,32.0,110.810558,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,26.0,100.857513,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
2,20.0,66.199017,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
3,18.0,77.017056,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,26.0,137.221167,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0


In [25]:
var = 'wiek'
bins=30
target_0 = xy_klienci_smote.loc[xy_klienci_smote[target] == 0]
hist, edges = np.histogram(target_0[var], density=True, bins=bins)
p1 = figure(title = "Rozkład zmiennej: "+var+" dla "+target+"=0", plot_width=400, plot_height=400)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p1.y_range.start = 0

target_1 = xy_klienci_smote.loc[xy_klienci_smote[target] == 1]
hist, edges = np.histogram(target_1[var], density=True, bins=bins)
p2 = figure(title = "Rozkład zmiennej: "+var+" dla "+target+"=1", plot_width=400, plot_height=400)
p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p2.y_range.start = 0

show(row(p1, p2))

In [26]:
var = 'kwota'
bins=30
target_0 = xy_klienci_smote.loc[xy_klienci_smote[target] == 0]
hist, edges = np.histogram(target_0[var], density=True, bins=bins)
p1 = figure(title = "Rozkład zmiennej: "+var+" dla "+target+"=0", plot_width=400, plot_height=400)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p1.y_range.start = 0

target_1 = xy_klienci_smote.loc[xy_klienci_smote[target] == 1]
hist, edges = np.histogram(target_1[var], density=True, bins=bins)
p2 = figure(title = "Rozkład zmiennej: "+var+" dla "+target+"=1", plot_width=400, plot_height=400)
p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p2.y_range.start = 0

show(row(p1, p2))

In [27]:
def split_var(dataframe, col, tar):
    x = dataframe[col].to_numpy()
    y = dataframe[tar].to_numpy()
    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    model = DecisionTreeClassifier(max_depth=1, min_samples_split=1000) 
    model.fit(x, y)     
    return model.tree_.threshold[0]

In [28]:
split_wiek = split_var(xy_klienci_smote, 'wiek', target)
split_kwota = split_var(xy_klienci_smote, 'kwota', target)
print('Podział zmiennej wiek w punkcie: ' + str(split_wiek))
print('Podział zmiennej kwota w punkcie: ' + str(split_kwota))

Podział zmiennej wiek w punkcie: 44.999921798706055
Podział zmiennej kwota w punkcie: 50.227338790893555


In [29]:
klienci['wiek_cat'] = np.where(klienci['wiek']<= split_wiek, 'młody', 'starszy')
klienci['kwota_cat'] = np.where(klienci['kwota']<= split_kwota, 'mniejsze', 'większe')
klienci.drop(num_col, axis=1, inplace=True)
klienci.head()

,plec,wyksztalcenie,miejsce,wojewodztwo,wysylka_oferty,zakup,wiek_cat,kwota_cat
numer,,,,,,,,
1,k,srednie,wies,dolsl,nie,0,młody,większe
2,k,wyzsze,m100_500,zachpom,nie,0,młody,większe
3,m,srednie,m100_500,zachpom,nie,0,młody,większe
4,k,policealne,m100_500,sląs,tak,0,młody,większe
5,k,srednie,m100_500,zachpom,nie,0,młody,większe


In [30]:
klienci_final['wiek_cat'] = np.where(klienci_final['wiek']<= split_wiek, 'młody', 'starszy')
klienci_final['kwota_cat'] = np.where(klienci_final['kwota']<= split_kwota, 'mniejsze', 'większe')
klienci_final.drop(num_col, axis=1, inplace=True)
klienci_final.head()

,plec,wyksztalcenie,miejsce,wojewodztwo,wiek_cat,kwota_cat
numer,,,,,,
1,m,podstawowe,wies,zachpom,młody,większe
2,k,podstawowe,wies,dolsl,młody,mniejsze
3,m,srednie,m10_100,dolsl,młody,większe
4,m,policealne,m100_500,dolsl,młody,większe
5,k,srednie,m_500,dolsl,młody,większe


In [31]:
cat_col = ['wiek_cat', 'plec', 'wyksztalcenie', 'miejsce','kwota_cat', 'wojewodztwo', 'wysylka_oferty']

In [32]:
for i in cat_col: 
    tab1 = mean_by(klienci, i, target)
    label = tab1.index.values
    values = tab1['zakup']  
    p = figure(x_range=label, plot_height=250, title="Prawdopodobieństwo zakupu względem kategorii zmiennej: "+ i)
    p.vbar(x=label, top=values, width=0.9)
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    show(p)

In [33]:
klienci_marketing = klienci.loc[klienci['wysylka_oferty'] == 'tak']
klienci_marketing.drop('wysylka_oferty', axis=1, inplace=True)
klienci_marketing.head()

,plec,wyksztalcenie,miejsce,wojewodztwo,zakup,wiek_cat,kwota_cat
numer,,,,,,,
4,k,policealne,m100_500,sląs,0,młody,większe
8,m,podstawowe,m0_10,dolsl,1,młody,mniejsze
22,m,srednie,m100_500,dolsl,1,młody,większe
23,k,podstawowe,m0_10,zachpom,0,młody,większe
30,m,srednie,wies,dolsl,1,młody,większe


In [34]:
klienci_marketing[target].value_counts()

0    1280
1     726
Name: zakup, dtype: int64

In [35]:
cat_col = ['wiek_cat', 'plec', 'wyksztalcenie', 'miejsce','kwota_cat', 'wojewodztwo']
X = klienci_marketing[cat_col]
Y = klienci_marketing[target]

In [36]:
X = pd.get_dummies(X, columns = cat_col, prefix = cat_col)

In [37]:
seed = 42
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

Typowanie klientów skłonnych do zakupu to otrzymaniu oferty

In [38]:
model = XGBClassifier(n_estimators=5000, max_depth=8, seed=seed, eval_metric = "auc", nthread=8)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=5000, n_jobs=1,
              nthread=8, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=None, subsample=1, verbosity=1)

In [39]:
predictions = model.predict(X_test)

In [40]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 69.27%


In [41]:
cat_col = ['wiek_cat', 'plec', 'wyksztalcenie', 'miejsce','kwota_cat', 'wojewodztwo']
X_score = klienci[cat_col]
X_score = pd.get_dummies(X_score, columns = cat_col, prefix = cat_col)
Y_score = model.predict(X_score)
klienci['profil_marketing'] = Y_score
klienci.drop('wysylka_oferty', axis=1, inplace=True)
dict = {1 : 'tak', 0: 'nie'} 
klienci['profil_marketing'].replace(dict, inplace=True)
klienci.head()

,plec,wyksztalcenie,miejsce,wojewodztwo,zakup,wiek_cat,kwota_cat,profil_marketing
numer,,,,,,,,
1,k,srednie,wies,dolsl,0,młody,większe,tak
2,k,wyzsze,m100_500,zachpom,0,młody,większe,nie
3,m,srednie,m100_500,zachpom,0,młody,większe,tak
4,k,policealne,m100_500,sląs,0,młody,większe,nie
5,k,srednie,m100_500,zachpom,0,młody,większe,tak


In [42]:
rows_number(klienci)

Liczba rekordów w zbiorze: 10034


In [43]:
cat_col = ['wiek_cat', 'plec', 'wyksztalcenie', 'miejsce','kwota_cat', 'wojewodztwo']
X_score = klienci_final[cat_col]
X_score = pd.get_dummies(X_score, columns = cat_col, prefix = cat_col)
Y_score = model.predict(X_score)
klienci_final['profil_marketing'] = Y_score
dict = {1 : 'tak', 0: 'nie'} 
klienci_final['profil_marketing'].replace(dict, inplace=True)
klienci_final.head()

,plec,wyksztalcenie,miejsce,wojewodztwo,wiek_cat,kwota_cat,profil_marketing
numer,,,,,,,
1,m,podstawowe,wies,zachpom,młody,większe,nie
2,k,podstawowe,wies,dolsl,młody,mniejsze,nie
3,m,srednie,m10_100,dolsl,młody,większe,tak
4,m,policealne,m100_500,dolsl,młody,większe,nie
5,k,srednie,m_500,dolsl,młody,większe,tak


In [44]:
rows_number(klienci_final)

Liczba rekordów w zbiorze: 100895


In [45]:
klienci_copy = klienci.copy()
klienci_final_copy = klienci_final.copy()

In [46]:
klienci[target].value_counts()

0    8717
1    1317
Name: zakup, dtype: int64

In [47]:
class_0 = klienci.loc[klienci[target] == 0]
class_1 = klienci.loc[klienci[target] == 1]
class_0_count = class_0.shape[0]
class_1_count = class_1.shape[0]
n_1 = class_0_count - class_1_count
print('0='+str(class_0_count)+', 1='+str(class_1_count)+', N1='+str(n_1))

index_class_1 = class_1.index.tolist()
class_1_new = choices(index_class_1, k=n_1)
index_class_1_final = index_class_1 + class_1_new
index_class_0 = class_0.index.tolist()
index_class_final = index_class_0 + index_class_1_final
n = len(index_class_final)
print('N='+str(n))

index_set = pd.DataFrame(index=index_class_final)
klienci_oversampling = index_set.join(klienci)

0=8717, 1=1317, N1=7400
N=17434


In [48]:
print(klienci_oversampling[target].value_counts())
print(klienci_oversampling.shape[0])

1    8717
0    8717
Name: zakup, dtype: int64
17434


In [49]:
cat_col = ['plec', 'wyksztalcenie','miejsce', 'wojewodztwo', 'wiek_cat', 'kwota_cat', 'profil_marketing']
klienci_oversampling_encoding = klienci_oversampling
klienci_oversampling_encoding.reset_index(drop=True, inplace=True)
klienci_copy_encoding = klienci_copy
klienci_final_copy_encoding = klienci_final_copy

for i in cat_col: 
  WOE_encoder = WOEEncoder()
  woe_encoding = WOE_encoder.fit_transform(klienci_oversampling[i], klienci_oversampling[target])
  woe_encoding = pd.DataFrame(woe_encoding)
  new_name = 'woe_' + i
  woe_encoding.columns = [new_name]
  klienci_oversampling_encoding = klienci_oversampling_encoding.join(woe_encoding)

  woe_encoding_score = WOE_encoder.transform(klienci_copy_encoding[i])
  woe_encoding_score = pd.DataFrame(woe_encoding_score)
  woe_encoding_score.columns = [new_name]
  klienci_copy_encoding = klienci_copy_encoding.join(woe_encoding_score)
  
  woe_encoding_score = WOE_encoder.transform(klienci_final_copy_encoding[i])
  woe_encoding_score = pd.DataFrame(woe_encoding_score)
  woe_encoding_score.columns = [new_name]
  klienci_final_copy_encoding = klienci_final_copy_encoding.join(woe_encoding_score)

In [50]:
display(klienci_oversampling_encoding)

,plec,wyksztalcenie,miejsce,wojewodztwo,zakup,wiek_cat,kwota_cat,profil_marketing,woe_plec,woe_wyksztalcenie,woe_miejsce,woe_wojewodztwo,woe_wiek_cat,woe_kwota_cat,woe_profil_marketing
0,k,srednie,wies,dolsl,0,młody,większe,tak,-0.024716,-0.146720,-1.295834,0.025383,0.412572,0.103456,0.518902
1,k,wyzsze,m100_500,zachpom,0,młody,większe,nie,-0.024716,1.318825,0.562018,-0.049643,0.412572,0.103456,-0.465330
2,m,srednie,m100_500,zachpom,0,młody,większe,tak,0.014305,-0.146720,0.562018,-0.049643,0.412572,0.103456,0.518902
3,k,policealne,m100_500,sląs,0,młody,większe,nie,-0.024716,-2.052773,0.562018,-0.125915,0.412572,0.103456,-0.465330
4,k,srednie,m100_500,zachpom,0,młody,większe,tak,-0.024716,-0.146720,0.562018,-0.049643,0.412572,0.103456,0.518902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17429,k,srednie,wies,mazow,1,starszy,mniejsze,tak,-0.024716,-0.146720,-1.295834,-0.251642,-0.646184,-0.249137,0.518902
17430,k,srednie,wies,mazow,1,starszy,mniejsze,tak,-0.024716,-0.146720,-1.295834,-0.251642,-0.646184,-0.249137,0.518902
17431,k,srednie,wies,mazow,1,starszy,mniejsze,tak,-0.024716,-0.146720,-1.295834,-0.251642,-0.646184,-0.249137,0.518902
17432,k,srednie,wies,mazow,1,starszy,mniejsze,tak,-0.024716,-0.146720,-1.295834,-0.251642,-0.646184,-0.249137,0.518902


In [51]:
klienci_oversampling_encoding_no_cat = klienci_oversampling_encoding.drop(cat_col, axis=1)
klienci_copy_encoding_no_cat = klienci_copy_encoding.drop(cat_col, axis=1)
klienci_final_copy_encoding_no_cat = klienci_final_copy_encoding.drop(cat_col, axis=1)

In [52]:
rows_number(klienci_oversampling)
rows_number(klienci_oversampling_encoding_no_cat)
rows_number(klienci)
rows_number(klienci_copy_encoding_no_cat)
rows_number(klienci_final)
rows_number(klienci_final_copy_encoding_no_cat)

Liczba rekordów w zbiorze: 17434
Liczba rekordów w zbiorze: 17434
Liczba rekordów w zbiorze: 10034
Liczba rekordów w zbiorze: 10034
Liczba rekordów w zbiorze: 100895
Liczba rekordów w zbiorze: 100895


In [53]:
y_klienci_copy_encoding_no_cat = klienci_copy_encoding_no_cat[target]
X_klienci_copy_encoding_no_cat = klienci_copy_encoding_no_cat.drop(target, axis=1)

In [54]:
seed = 42
test_size = 0.4
y = klienci_oversampling_encoding_no_cat[target]
X = klienci_oversampling_encoding_no_cat.drop(target,axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [55]:
X_train.head()

,woe_plec,woe_wyksztalcenie,woe_miejsce,woe_wojewodztwo,woe_wiek_cat,woe_kwota_cat,woe_profil_marketing
4217,0.014305,-0.146720,0.562018,0.025383,0.412572,0.103456,0.518902
4242,0.014305,-0.146720,0.310509,-0.049643,0.412572,0.103456,0.518902
12983,0.014305,1.318825,0.808084,0.025383,0.412572,0.103456,-0.465330
17157,-0.024716,-2.776427,-1.295834,0.138150,-0.646184,-0.249137,-0.465330
16682,0.014305,-2.776427,-1.295834,-0.251642,-0.646184,0.103456,-0.465330


In [56]:
print(y_test.value_counts())

1    3518
0    3456
Name: zakup, dtype: int64


In [57]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
y_predict = model.predict(X_test)

In [59]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 71.62%


In [60]:
model = svm.SVC(kernel='rbf', C=1.0, gamma='auto')
model.fit(X, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [61]:
y_predict = model.predict(X_test)

In [62]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 71.59%


In [63]:
model = DecisionTreeClassifier() 
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [64]:
y_predict = model.predict(X_test)

In [65]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 74.65%


In [66]:
model = XGBClassifier(n_estimators=5000, max_depth=8, seed=seed, eval_metric = "auc", nthread=8)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=5000, n_jobs=1,
              nthread=8, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=None, subsample=1, verbosity=1)

In [67]:
y_predict = model.predict(X_test)

In [68]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.31%


In [69]:
model = XGBClassifier(n_estimators=5000, max_depth=8, seed=seed, eval_metric = "auc", nthread=8)
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=5000, n_jobs=1,
              nthread=8, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=None, subsample=1, verbosity=1)

In [70]:
y_predict = model.predict(X)

In [71]:
accuracy = accuracy_score(y, y_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.70%


In [72]:
y_klienci_predict = model.predict(X_klienci_copy_encoding_no_cat)

In [73]:
print(classification_report(y_klienci_copy_encoding_no_cat, y_klienci_predict)) 

              precision    recall  f1-score   support

           0       0.97      0.71      0.82      8717
           1       0.30      0.83      0.44      1317

    accuracy                           0.73     10034
   macro avg       0.63      0.77      0.63     10034
weighted avg       0.88      0.73      0.77     10034



In [74]:
y_klienci_predict_proba = model.predict_proba(X_klienci_copy_encoding_no_cat)
klienci_profit = klienci_copy_encoding_no_cat
klienci_profit['zakup_predict'] = y_klienci_predict
klienci_profit['zakup_proba'] = y_klienci_predict_proba[:, 1]

In [75]:
przychod = 100
koszt = -10
dochod = przychod + koszt
klienci_profit.loc[(klienci_profit[target] == 1)  & (klienci_profit['zakup_predict'] == 1), 'zysk'] = dochod
klienci_profit.loc[(klienci_profit[target] == 1)  & (klienci_profit['zakup_predict'] == 0), 'zysk'] = dochod * (-1)
klienci_profit.loc[(klienci_profit[target] == 0)  & (klienci_profit['zakup_predict'] == 1), 'zysk'] = koszt
klienci_profit.loc[(klienci_profit[target] == 0)  & (klienci_profit['zakup_predict'] == 0), 'zysk'] = 0
suma = klienci_profit['zysk'].sum()
display(klienci_profit)
print('Łączny zysk przed optymalizacją: $'+str(suma))

,zakup,woe_plec,woe_wyksztalcenie,woe_miejsce,woe_wojewodztwo,woe_wiek_cat,woe_kwota_cat,woe_profil_marketing,zakup_predict,zakup_proba,zysk
numer,,,,,,,,,,,
1,0,-0.024716,-0.146720,-1.295834,0.025383,0.412572,0.103456,0.518902,1,0.627259,-10.0
2,0,-0.024716,1.318825,0.562018,-0.049643,0.412572,0.103456,-0.465330,1,0.750764,-10.0
3,0,0.014305,-0.146720,0.562018,-0.049643,0.412572,0.103456,0.518902,1,0.637240,-10.0
4,0,-0.024716,-2.052773,0.562018,-0.125915,0.412572,0.103456,-0.465330,0,0.000001,0.0
5,0,-0.024716,-0.146720,0.562018,-0.049643,0.412572,0.103456,0.518902,1,0.557336,-10.0
...,...,...,...,...,...,...,...,...,...,...,...
10030,0,0.014305,-0.146720,-0.170636,-0.269982,-0.646184,-0.249137,-0.465330,0,0.000102,0.0
10031,0,0.014305,-2.776427,-1.295834,-0.049643,-0.646184,0.103456,-0.465330,0,0.000159,0.0
10032,0,0.014305,-0.146720,-1.295834,-0.251642,-0.646184,-0.249137,0.518902,0,0.458836,0.0


Łączny zysk przed optymalizacją: $53860.0


In [76]:
minp = klienci_profit['zakup_proba'].min()
maxp = klienci_profit['zakup_proba'].max()
rangep = np.arange(minp, maxp, 0.0001).tolist()

In [77]:
max_zysk = 0
cut_off = 0.0

for p in rangep: 
    klienci_profit['zakup_predict'] = np.where(klienci_profit['zakup_proba']>=p, 1, 0) 
    przychod = 100
    koszt = -10
    dochod = przychod + koszt
    klienci_profit.loc[(klienci_profit[target] == 1)  & (klienci_profit['zakup_predict'] == 1), 'zysk'] = dochod
    klienci_profit.loc[(klienci_profit[target] == 1)  & (klienci_profit['zakup_predict'] == 0), 'zysk'] = dochod * (-1)
    klienci_profit.loc[(klienci_profit[target] == 0)  & (klienci_profit['zakup_predict'] == 1), 'zysk'] = koszt
    klienci_profit.loc[(klienci_profit[target] == 0)  & (klienci_profit['zakup_predict'] == 0), 'zysk'] = 0
    suma = klienci_profit['zysk'].sum()
    if suma > max_zysk:
        max_zysk = suma
        cut_off = p
    else:
        pass
    
print('$'+str(max_zysk))
print('Cut-off: '+str(cut_off))

$70210.0
Cut-off: 0.21030000239444843


In [78]:
y_klienci_predict_proba = model.predict_proba(X_klienci_copy_encoding_no_cat)
klienci_profit = klienci_copy_encoding_no_cat
klienci_profit['zakup_proba'] = y_klienci_predict_proba[:, 1]
klienci_profit['zakup_predict'] = np.where(klienci_profit['zakup_proba']>=cut_off, 1, 0)

In [79]:
przychod = 100
koszt = -10
dochod = przychod + koszt
klienci_profit.loc[(klienci_profit[target] == 1)  & (klienci_profit['zakup_predict'] == 1), 'zysk'] = dochod
klienci_profit.loc[(klienci_profit[target] == 1)  & (klienci_profit['zakup_predict'] == 0), 'zysk'] = dochod * (-1)
klienci_profit.loc[(klienci_profit[target] == 0)  & (klienci_profit['zakup_predict'] == 1), 'zysk'] = koszt
klienci_profit.loc[(klienci_profit[target] == 0)  & (klienci_profit['zakup_predict'] == 0), 'zysk'] = 0
suma = klienci_profit['zysk'].sum()
display(klienci_profit)
print('Łączny zysk po optymalizacji: $'+str(suma))

,zakup,woe_plec,woe_wyksztalcenie,woe_miejsce,woe_wojewodztwo,woe_wiek_cat,woe_kwota_cat,woe_profil_marketing,zakup_predict,zakup_proba,zysk
numer,,,,,,,,,,,
1,0,-0.024716,-0.146720,-1.295834,0.025383,0.412572,0.103456,0.518902,1,0.627259,-10.0
2,0,-0.024716,1.318825,0.562018,-0.049643,0.412572,0.103456,-0.465330,1,0.750764,-10.0
3,0,0.014305,-0.146720,0.562018,-0.049643,0.412572,0.103456,0.518902,1,0.637240,-10.0
4,0,-0.024716,-2.052773,0.562018,-0.125915,0.412572,0.103456,-0.465330,0,0.000001,0.0
5,0,-0.024716,-0.146720,0.562018,-0.049643,0.412572,0.103456,0.518902,1,0.557336,-10.0
...,...,...,...,...,...,...,...,...,...,...,...
10030,0,0.014305,-0.146720,-0.170636,-0.269982,-0.646184,-0.249137,-0.465330,0,0.000102,0.0
10031,0,0.014305,-2.776427,-1.295834,-0.049643,-0.646184,0.103456,-0.465330,0,0.000159,0.0
10032,0,0.014305,-0.146720,-1.295834,-0.251642,-0.646184,-0.249137,0.518902,1,0.458836,-10.0


Łączny zysk po optymalizacji: $70210.0


In [80]:
klienci_final_proba = model.predict_proba(klienci_final_copy_encoding_no_cat)
klienci_final_proba_score = klienci_final_copy_encoding_no_cat
klienci_final_proba_score['zakup_proba'] = klienci_final_proba[:, 1]
klienci_final_proba_score['zakup_predict'] = np.where(klienci_final_proba_score['zakup_proba']>=cut_off, 1, 0)
display(klienci_final_proba_score)

,woe_plec,woe_wyksztalcenie,woe_miejsce,woe_wojewodztwo,woe_wiek_cat,woe_kwota_cat,woe_profil_marketing,zakup_proba,zakup_predict
numer,,,,,,,,,
1,0.014305,-2.776427,-1.295834,-0.049643,0.412572,0.103456,-0.465330,0.000035,0
2,-0.024716,-2.776427,-1.295834,0.025383,0.412572,-0.249137,-0.465330,0.112541,0
3,0.014305,-0.146720,0.310509,0.025383,0.412572,0.103456,0.518902,0.563387,1
4,0.014305,-2.052773,0.562018,0.025383,0.412572,0.103456,-0.465330,0.005090,0
5,-0.024716,-0.146720,0.808084,0.025383,0.412572,0.103456,0.518902,0.485192,1
...,...,...,...,...,...,...,...,...,...
100891,0.014305,-0.146720,-0.170636,-0.049643,-0.646184,-0.249137,-0.465330,0.486166,1
100892,0.014305,-0.146720,-1.295834,0.025383,-0.646184,0.103456,-0.465330,0.210246,0
100893,0.014305,-2.752655,-1.295834,0.025383,-0.646184,0.103456,-0.465330,0.165468,0


In [81]:
output = '"' + klienci_final_proba_score.index.astype(str) + '",' + klienci_final_proba_score['zakup_predict'].astype(str)
output = pd.DataFrame(output)
output.columns = ['x']
csv.QUOTE_NONE
output.to_csv('output.txt', header=True, index=False, sep='\t', mode='w', quoting=csv.QUOTE_NONE)